# Section 3: From Classical FFT to Quantum QFT – Transforming Information

**Objective:** Understand what a Fourier Transform does intuitively and appreciate the power of its quantum counterpart (Quantum Fourier Transform - QFT) without complex math, specifically in contexts where it offers a quantum advantage.

---

### Key Concepts: The Power of Frequency Analysis

* **What is a Fourier Transform (Classical)?**
    * **Intuition:** Imagine a complex musical chord playing. Your ear can often pick out the individual notes. A Fourier Transform does something similar for any complex signal.
    * It's a "frequency analyzer" that takes a signal (which might vary over time or space) and decomposes it into its constituent simple sine and cosine waves (its "frequencies") and their corresponding amplitudes and **phases**.
    * **Importance in Telecom:** The Fourier Transform is fundamental in classical signal processing and telecommunications. It's used everywhere from signal analysis (understanding what frequencies are present in a signal), to modulation/demodulation (encoding/decoding information onto carrier waves), filtering noise, image compression, and much more. It allows us to move between time/space domains and frequency domains.

* **Why Quantum Fourier Transform (QFT)?**
    * **Intuition:** The QFT performs this "frequency analysis" not on classical signals, but on *quantum states*. It can efficiently extract information about the periodicities (or frequencies) encoded within the phases of a quantum superposition state.
    * **Power:** For certain problems, the QFT can perform this analysis exponentially faster than any known classical algorithm (the Fast Fourier Transform - FFT). However, it's crucial to understand *when* this advantage applies, as it's not a direct drop-in replacement for all classical FFT uses.
    * **Fundamental Subroutine:** It's a key building block within some of the most powerful quantum algorithms, such as Shor's algorithm for factoring large numbers (which has profound implications for breaking many modern cryptographic schemes) and the Quantum Phase Estimation algorithm.

* **Complex Numbers (Very Basic Intuition):**
    * To understand waves and quantum states, we often use **complex numbers** (numbers like `a + bi`, where `i` is the imaginary unit, $\sqrt{-1}$). 
    * **Intuition:** Think of a complex number not just as a point on a 2D plane, but as a way to represent something with both a *magnitude* (how "strong" it is) and a *phase* (its "angle" or position in a cycle).
    * For waves, phase tells you where in its cycle the wave is at a given point. For quantum states, the phase of a qubit's amplitudes is crucial for allowing quantum interference to happen. The QFT works by manipulating these phases precisely.

---

### Implementation: From Classical FFT to Conceptual QFT Circuit

#### 1. Classical FFT (Brief Demo)

Let's see a super simple example of a classical Fast Fourier Transform (FFT) using NumPy. We'll take a very basic signal and see its frequency components.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# A simple classical signal (e.g., a short pulse, or two superimposed sine waves)
# Let's create a signal that's a mix of two frequencies
sampling_rate = 100 # Hz
duration = 1      # seconds
t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)

# Signal with two components: 5 Hz and 15 Hz
signal = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 15 * t)

# Perform the Fast Fourier Transform (FFT)
fft_output = np.fft.fft(signal)
frequencies = np.fft.fftfreq(len(signal), d=1/sampling_rate)

# We usually only care about the positive frequencies for real signals
positive_frequencies = frequencies[frequencies >= 0]
positive_fft_output = 2/len(signal) * np.abs(fft_output[frequencies >= 0]) # Normalize and take magnitude

# Plot the original signal
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(t, signal)
plt.title('Original Classical Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot the FFT results (frequency domain)
plt.subplot(1, 2, 2)
plt.stem(positive_frequencies, positive_fft_output, basefmt=" ")
plt.title('Frequency Spectrum (FFT)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim(-1, sampling_rate / 2) # Show relevant frequency range
plt.grid(True)
plt.tight_layout()
plt.show()

print("Notice how the FFT clearly shows peaks at the 5 Hz and 15 Hz frequencies that make up the signal.")

#### 2. Quantum Fourier Transform (QFT) Circuit (General Structure)

The QFT circuit is built using two primary types of gates:
* **Hadamard gates (H):** Used to create superpositions.
* **Controlled-Phase gates (`cp` or `rz` with `cx`):** These are the core for phase manipulation. A `cp(theta)` gate applies a phase rotation `e^(i*theta)` to the target qubit *only if* the control qubit is in the $|1\rangle$ state. The angle `theta` depends on the position of the control qubit relative to the target qubit in the circuit.
* **SWAP gates:** Often placed at the end to reverse the order of qubits, as the QFT typically outputs the transformed state in reverse order of its input.

Here's a general function for an N-qubit QFT circuit. The number of qubits can be adjusted, and the circuit will scale accordingly.

```python
def qft_circuit(qc, n_qubits):
    for i in range(n_qubits):
        qc.h(i) # Apply Hadamard to the current qubit
        # Apply controlled-phase rotations with preceding qubits
        for j in range(i + 1, n_qubits):
            # The angle decreases for qubits further away
            # Formula for angle: pi / (2**(j-i))
            qc.cp(np.pi / (2**(j - i)), i, j) # cp(angle, control, target)
    
    # Apply SWAP gates to reverse the qubit order (if needed for standard output convention)
    for i in range(n_qubits // 2):
        qc.swap(i, n_qubits - 1 - i)
```

**Important Note on QFT Output Measurement:**

When the QFT is applied to a single computational basis state (like $|000\rangle$, $|001\rangle$, etc.), the resulting state is a superposition where the *magnitudes* of all basis state amplitudes are equal. This means if you simply measure the qubits after applying QFT to such an input, you will observe a **nearly uniform distribution** across all possible outcomes. The power of QFT isn't in making a single measurement outcome highly probable for an arbitrary input state, but in encoding information into the *phases* of the quantum state, which can then be exploited by subsequent quantum operations.

Let's see this in action for an initial state of $|000\rangle$.

In [ ]:
%matplotlib inline
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

# Function to apply QFT for n qubits (as defined above)
def qft_circuit(qc, n_qubits):
    for i in range(n_qubits):
        qc.h(i)
        for j in range(i + 1, n_qubits):
            qc.cp(np.pi / (2**(j - i)), i, j)
    
    for i in range(n_qubits // 2):
        qc.swap(i, n_qubits - 1 - i)

# Set the number of qubits for this QFT demonstration
num_qft_qubits = 3 

# Create a quantum circuit starting with all |0>s
qc_qft_uniform_demo = QuantumCircuit(num_qft_qubits, num_qft_qubits)

qc_qft_uniform_demo.barrier() # Visual separator

# Apply the QFT transformation
qft_circuit(qc_qft_uniform_demo, num_qft_qubits)

qc_qft_uniform_demo.barrier() # Visual separator

# Measure all qubits
qc_qft_uniform_demo.measure(range(num_qft_qubits), range(num_qft_qubits))

# Draw the QFT circuit
print(f"{num_qft_qubits}-Qubit Quantum Fourier Transform (QFT) Circuit Diagram:")
qc_qft_uniform_demo.draw(output="mpl", fold=-1) 
plt.show()

# Select the Aer simulator backend
simulator = AerSimulator()

# Execute the circuit (for an input of |0...0>, the output should be uniformly distributed)
job = simulator.run(qc_qft_uniform_demo, shots=1024)
result = job.result()

# Get the measurement counts
counts_uniform = result.get_counts(qc_qft_uniform_demo)

print(f"\nMeasurement Counts (for initial state |{'0'*num_qft_qubits}>):")
print(counts_uniform)

# Plot the results
plot_histogram(counts_uniform).set_size_inches(8, 5)
plt.title(f"QFT Output Measurement Results (Initial state: |{'0'*num_qft_qubits}>)")
plt.show()

print(f"\nDiscussion: As expected, the outcomes are roughly uniformly distributed. This is because the QFT of the |{'0'*num_qft_qubits}> state is an equal superposition of all {2**num_qft_qubits} basis states. While this doesn't immediately show 'peaks', it's a foundational step to understanding its power in more complex scenarios.")

---

### Demonstrating QFT's Power: Detecting Hidden Frequencies/Patterns

The true power of the QFT isn't in making a single outcome more probable when applied to an arbitrary single computational basis state. Its strength lies in its ability to efficiently transform **superposition states that have an underlying periodicity or a hidden phase structure** into a form where direct measurement *does* reveal distinct peaks.

**Comparing to Classical FFT in Telecom:**
* **Classical FFT:** Excellent for analyzing known signals or signals where frequencies are somewhat isolated. It processes data sequentially.
* **Quantum QFT:** The quantum advantage comes from its ability to efficiently extract hidden information (like periods or specific 'frequencies') from superpositions. It leverages quantum parallelism and interference to analyze many possibilities simultaneously and amplify the 'correct' frequency outcomes while canceling out others.

**Potential Telecom Applications Where QFT-related Algorithms Could Offer Improvements:**

1.  **Blind Signal Separation and Source Localization:** In crowded spectrums, separating mixed signals from different users or pinpointing their locations is computationally intensive. QFT's ability to efficiently find hidden periodicities or phase relationships in a quantum representation of these signals could lead to faster, more accurate separation and localization.
2.  **Advanced Channel Estimation and Equalization:** For complex and rapidly changing wireless channels, estimating distortion parameters is crucial. QFT-based algorithms (like Quantum Phase Estimation) might estimate these parameters with higher precision or faster than classical methods, leading to more robust and higher-throughput links.
3.  **Complex Interference Cancellation:** Identifying and canceling intricate, periodic interference patterns is vital. QFT could efficiently pinpoint these patterns, enabling more precise and faster cancellation, improving Signal-to-Noise Ratio (SNR).
4.  **Enhanced Radar/Lidar Signal Processing:** In advanced sensing, extracting precise range, velocity, and angular information from complex signals is demanding. QFT-based methods could potentially enhance the resolution and speed of target detection and parameter estimation by exploiting subtle phase information.

### Hands-on Demonstration: Finding a Simulated Hidden Pattern

Let's demonstrate how QFT reveals hidden patterns. We will prepare a specific {num_qft_qubits}-qubit input state that inherently has a **simulated periodicity**. When we apply the QFT to this state, you will see clear peaks in the measurement results, demonstrating its power in detecting these underlying 'frequencies'.

For this example, we'll keep `num_qft_qubits` at 3. We will prepare the input state $\frac{1}{\sqrt{2}} (|000\rangle + |010\rangle)$. Think of this as a simplified 'signal' where the pattern `010` is present alongside `000` – implying a repeating characteristic or 'period' related to the difference between these states.

In [ ]:
%matplotlib inline
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

# Function to apply QFT for n qubits (as defined previously)
def qft_circuit(qc, n_qubits):
    for i in range(n_qubits):
        qc.h(i)
        for j in range(i + 1, n_qubits):
            qc.cp(np.pi / (2**(j - i)), i, j)
    
    for i in range(n_qubits // 2):
        qc.swap(i, n_qubits - 1 - i)

# Set the number of qubits for this specific QFT demonstration
# (Note: The periodic state preparation below is tailored for 3 qubits)
num_qft_qubits = 3 

# Create a quantum circuit
qc_qft_periodic_demo = QuantumCircuit(num_qft_qubits, num_qft_qubits)

# --- Prepare the periodic input state: 1/sqrt(2) * (|000> + |010>) ---
# This is achieved by applying a Hadamard gate to qubit 1 (the middle qubit)
# Initial state: |000>
# After H(1): |0>_q2 * (1/sqrt(2)(|0>+|1>))_q1 * |0>_q0 = 1/sqrt(2) * (|000> + |010>)
qc_qft_periodic_demo.h(1) 
qc_qft_periodic_demo.barrier() # Visual separator

# Apply the QFT transformation
qft_circuit(qc_qft_periodic_demo, num_qft_qubits)

qc_qft_periodic_demo.barrier() # Visual separator

# Measure all qubits
qc_qft_periodic_demo.measure(range(num_qft_qubits), range(num_qft_qubits))

# Draw the QFT circuit
print(f"{num_qft_qubits}-Qubit QFT Circuit Diagram for Periodic Input:")
qc_qft_periodic_demo.draw(output="mpl", fold=-1) 
plt.show()

# Select the Aer simulator backend
simulator = AerSimulator()

# Execute the circuit
job = simulator.run(qc_qft_periodic_demo, shots=1024)
result = job.result()

# Get the measurement counts
counts_periodic = result.get_counts(qc_qft_periodic_demo)

print(f"\nMeasurement Counts (for initial periodic state |000> + |010>):")
print(counts_periodic)

# Plot the results
plot_histogram(counts_periodic).set_size_inches(8, 5)
plt.title(f"QFT Output Measurement Results (Initial state: |000> + |010>)")
plt.show()

print("\nObservation: Notice how the outcomes are now predominantly concentrated around '000' and '100'. This demonstrates QFT's ability to transform a state with a hidden periodicity (related to a period of 2, or 010) into a state where these 'frequencies' (0 and 4, or 000 and 100) become highly probable upon measurement. This is the core mechanism enabling quantum algorithms to solve problems faster than classical computers, such as finding periods in Shor's algorithm, and potentially for complex signal processing in telecommunications.")